In [1]:
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions

/home/s1928563/programs/miniconda3/envs/adbench/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv1 = nn.Conv2d(
            in_channels=1,
            out_channels=6,
            kernel_size=4,
            stride=1,
            padding=0,
        )
        self.conv2 = nn.Conv2d(
            in_channels=6,
            out_channels=16,
            kernel_size=4,
            stride=1,
            padding=0,
        )
        self.conv3 = nn.Conv2d(
            in_channels=16,
            out_channels=144,
            kernel_size=4,
            stride=1,
            padding=0,
        )
        self.linear1 = nn.Linear(144, 84)
        self.linear2 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(
            self.conv3(x)
        )  # num_examples x 120 x 1 x 1 --> num_examples x 120
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.linear1(x))
        x = self.linear2(x)
        return x


In [3]:
# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 20

from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from tqdm import tqdm  # For nice progress bar!
model = LeNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [4]:
from data_preprocessing import get_datasets_validation
train_loader, val_loader, test_loader = get_datasets_validation("dataset", 32)

got test


In [5]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    """
    Check accuracy of our trained model given a loader and a model
    Parameters:
        loader: torch.utils.data.DataLoader
            A loader for the dataset you want to check accuracy on
        model: nn.Module
            The model you want to check accuracy on
    Returns:
        acc: float
            The accuracy of the model on the dataset given by the loader
    """
    num_correct = 0
    num_samples = 0
    model.eval()

    # We don't need to keep track of gradients here so we wrap it in torch.no_grad()
    with torch.no_grad():
        # Loop through the data
        for x, y in loader:
            # Move data to device
            x = x.to(device=device)
            y = y.to(device=device)
            # Get to correct shape
            # x = x.reshape(x.shape[0], -1)
            x = x.float()
            # Forward pass
            scores = model(x)
            _, predictions = scores.max(1)
            # Check how many we got correct
            num_correct += (predictions == y).sum()
            # Keep track of number of samples
            num_samples += predictions.size(0)
    model.train()
    return num_correct / num_samples

In [6]:
# Train Network

# Define some variables to keep track of the best model and validation accuracy
best_model = None
best_val_acc = 0.0
path = "lenet/lenet_50_epochs_64_batch_2.txt"

with open(path, "w") as f:
    for epoch in range(51):
        
        for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
            # Get data to cuda if possible
            data = data.to(device=device)
            # print(data.shape)
            targets = targets.to(device=device)
            # print(targets.shape)

            # Get to correct shape
            # data = data.reshape(data.shape[0], -1)
            # print(data.shape)
            data = data.float()
            # Forward
            scores = model(data)
            loss = criterion(scores, targets)
            # Backward
            optimizer.zero_grad()
            loss.backward()
            # Gradient descent or adam step
            optimizer.step()
        
        if epoch % 5 == 0 and epoch != 0:
            val_acc = check_accuracy(val_loader, model)
            train_acc = check_accuracy(train_loader, model)
            test_acc = check_accuracy(test_loader, model)
            # If the current model has a better validation accuracy, save it
            if val_acc > best_val_acc:
                best_model = model.state_dict()
                best_val_acc = val_acc

            # Check accuracy on training & test to see how good our model
            print("EPOCH: " + str(epoch))
            print("Loss: " + str(loss))
            print(f"Accuracy on training set: {train_acc*100:.2f}")
            print(f"Accuracy on validation set: {val_acc*100:.2f}")
            print(f"Accuracy on test set: {test_acc*100:.2f}")
            f.write("EPOCH " + str(epoch) + 
                    ", loss:"+str(loss) +
                    ", train_acc:" + str(train_acc) + 
                    ", val_acc:" + str(val_acc) +
                    ", test_acc:" + str(test_acc) + "\n")

100%|██████████| 6227/6227 [00:29<00:00, 211.64it/s]


EPOCH: 5
Loss: tensor(0.0052, grad_fn=<NllLossBackward0>)
Accuracy on training set: 96.91
Accuracy on validation set: 95.75
Accuracy on test set: 90.28


100%|██████████| 6227/6227 [00:37<00:00, 166.83it/s]


EPOCH: 10
Loss: tensor(0.1539, grad_fn=<NllLossBackward0>)
Accuracy on training set: 97.93
Accuracy on validation set: 96.21
Accuracy on test set: 90.71


100%|██████████| 6227/6227 [00:41<00:00, 149.48it/s]


...

EPOCH: 40
Loss: tensor(0.2519, grad_fn=<NllLossBackward0>)
Accuracy on training set: 99.05
Accuracy on validation set: 95.81
Accuracy on test set: 89.71


100%|██████████| 6227/6227 [00:44<00:00, 139.07it/s]


EPOCH: 45
Loss: tensor(0.0019, grad_fn=<NllLossBackward0>)
Accuracy on training set: 99.00
Accuracy on validation set: 95.75
Accuracy on test set: 89.82


100%|██████████| 6227/6227 [00:46<00:00, 135.03it/s]


EPOCH: 50
Loss: tensor(0.0007, grad_fn=<NllLossBackward0>)
Accuracy on training set: 98.97
Accuracy on validation set: 95.71
Accuracy on test set: 89.42


In [7]:
best_val_acc

tensor(0.9621)

In [10]:
def save_checkpoint(state, filename):
    name = filename + ".pth.tar"
    print("=> Saving checkpoint")
    torch.save(state, name)

# save model
checkpoint = {"state_dict": best_model, "optimizer": optimizer.state_dict()}
# Try save checkpoint
save_checkpoint(checkpoint, filename="lenet_model")

=> Saving checkpoint


In [11]:
### Inference time
import torch.nn as nn
inf = LeNet()
inf.linear1 = nn.Identity()
inf.linear2 = nn.Identity()

In [12]:
inf

LeNet(
  (relu): ReLU()
  (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv1): Conv2d(1, 6, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(4, 4), stride=(1, 1))
  (conv3): Conv2d(16, 144, kernel_size=(4, 4), stride=(1, 1))
  (linear1): Identity()
  (linear2): Identity()
)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### PLOT LENET BEST MODEL IDENTIFICATION
path = "lenet_50_epochs_64_batch.txt"
epochs = dict()
with open(path) as f:
    lines = f.readlines() # list containing lines of file
    epochs = []
    loss = []
    train_acc = []
    val_acc = []
    test_acc = []
    for l in lines:
        split_l = l.split(',')
        epoch = split_l[0].split(" ")[-1]
        if epoch not in epochs.keys():
            epochs[epoch] = ["train": [], "test": [], "val": []]

    for line in lines:
        split_l = l.split(',')
        epoch = split_l[0].split(" ")[-1]
        if epoch in epochs:
            acc = float(split_l[3].split(":")[1][7:-1])
            val = float(split_l[4].split(":")[1][7:-1])
            test = float(split_l[5].split(":")[1][7:-2])
            epochs[epoch]["train"].append(acc) 
            epochs[epoch]["test"].append(test) 
            epochs[epoch]["val"].append(val) 

fig, axs = plt.subplots()
default_x_ticks = range(len(epochs))
axs.set_xticks(default_x_ticks, epochs)

axs.plot(default_x_ticks,test_acc, label="Test set")
axs.plot(default_x_ticks,val_acc, label="Validation set")
axs.plot(default_x_ticks,train_acc, label="Train set")

axs.set_title("LeNet-5 Performance on All Datasets")
axs.set_ylabel("Score")
axs.set_xlabel("Epochs")

axs.legend()
plt.savefig("lenet.png")
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'lenet_50_epochs_64_batch.txt'